# Code Sandbox

A place to test new code and functions

In [1]:
# import packages and set working directory
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from math import exp
import seaborn as sns

#os.chdir('../maize-Toff')

# import objects
from farm import Climate
from farm import Soil
from farm import Crop
from farm import CropModel
from farm.functions import *

In [2]:
# new stuff for non-parametric approach for simulating rainfall

In [3]:
s0_climate = Climate(station='OL JOGI FARM', data_file='../data/CETRAD/CETRAD_rainfall.csv', interval='dekad')

FileNotFoundError: [Errno 2] File ../data/CETRAD/CETRAD_rainfall.csv does not exist: '../data/CETRAD/CETRAD_rainfall.csv'

In [ ]:
climate = Climate(station='OL JOGI FARM')

soil = Soil(texture='clay loam')
crop = Crop(soil=soil, lgp=180)
soil.set_nZr(crop)

model = CropModel(crop=crop, climate=climate, soil=soil)
model.run()
o = model.output() # This is the correct length
o.head(25)
o_ = o.loc[o['dos'] >= 1]
o_.head(20)
print(soil.sh, soil.sfc,  crop.s_star, crop.sw)

In [34]:
# playing around with dynamic stress

KPAR_CROP = 0.5
RPAR_CROP = 2
TGRW_CROP = 180
QPAR_CROP = 2

INVL_SIMU = 1 # actually really not sure about this

SWLT_SOIL = crop.sw
SSTR_SOIL = crop.s_star

## Define Function
def calc_dstress(s, stress): #self,  add this
    '''Calculates dyamic stress (theta) which is a measure of total water stress during the growing season
    as proposed in Porporato et al. (2001).
    
    Usage: calc_dstress(s, stress):
        s = relative saturation [0-1]
        stress = static stress [0-1]
        
    Arguments:
    average static stress = not sure about this: mstr_memb
    stress point T = average duration of an excursion below stress point
    n_star = number of excursions during the growing season
    T_seas = length of the growing season: TGRW_CROP
    k = fraction of growing season before crop fails = KPAR_CROP
    r = effect of number of excurs below strss point = RPAR_CROP
    '''
    # Calculate average static stress
    if len(stress) > 0:
        # It's annoying but need to subset the time series so it's just the growing season
        # Subset the growing period and get avg soil moisture
        start = 21 
        end = start + TGRW_CROP
        stress_subset = stress[start:end]
        mstr_memb = np.mean(stress_subset)
    else:
        mstr_memb = 0.

    # Calculate Crossing Parameters
    indx_memb = np.where(s >= SSTR_SOIL)
    ccrs_memb = np.diff(np.append(0, np.append(indx_memb, INVL_SIMU * TGRW_CROP + 1))) - 1
    ccrs_memb = ccrs_memb[ccrs_memb > 0]
    ncrs_memb = len(ccrs_memb)  # dim
    if ncrs_memb > 0:
        mcrs_memb = np.mean(ccrs_memb) / INVL_SIMU # days
    else:
        mcrs_memb = 0.
        
    # Calculate dynamic stress
    dstr_memb = ((mstr_memb * mcrs_memb) / (KPAR_CROP * TGRW_CROP))**(ncrs_memb**-RPAR_CROP)
    if dstr_memb > 1.:
        dstr_memb = 1.

    return(dstr_memb)


calc_dstress(s = o.s, stress = o.stress)

# convert this into yield
#4260*(calc_dstress(memb = o.s, stress = o.stress))


0.5618054554391123

In [10]:
# Use Drew's code to write a test

SWLT_SOIL = crop.sw
SSTR_SOIL = crop.s_star
QPAR_CROP = 2

    # i added this in    
# Calculate Average Static Stress -- I think I already do this myself and can just import this average value
sstr_memb = (SSTR_SOIL - o.s) / (SSTR_SOIL - SWLT_SOIL)  # dim
sstr_memb = sstr_memb[sstr_memb > 0.]
sstr_memb[sstr_memb > 1.] = 1.
if len(sstr_memb) > 0:
    mstr_memb = np.mean(sstr_memb**QPAR_CROP)  # dim
else:
    mstr_memb = 0.  # dim
    
    # Can write a test using Drew's code
    # above is Drew's code. If I just want to use the stress function I already calculcated, do:
    
mstr_memb

0.19388840256942838

In [4]:
average_soil_moisture(model, n_sims=1000, doy=90)

(59    0.555472
 dtype: float64,
 59    0.077929
 dtype: float64)